In [ ]:
import pandas as pd
#from google.colab.patches import cv2_imshow
import matplotlib.pyplot as plt
import tensorflow.keras.backend as K
import tensorflow as tf
import numpy as np
import cv2

from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense,Input,Lambda

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds
from functools import partial
from albumentations import (
    Compose, RandomBrightness, JpegCompression, HueSaturationValue, RandomContrast, HorizontalFlip,Normalize,RandomBrightnessContrast,
    VerticalFlip,
    Rotate
)
import pandas as pd
#from google.colab.patches import cv2_imshow
import matplotlib.pyplot as plt
import tensorflow.keras.backend as K
import tensorflow as tf
import numpy as np
import cv2

from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense,Input,Lambda

#from albumentations import *
AUTOTUNE = tf.data.experimental.AUTOTUNE

In [ ]:
testData = pd.read_csv("../input/mnist-zeroshot-learning/testMnist.csv")
testPictureIndex = testData.index.values
testTensor = tf.convert_to_tensor(testData.iloc[:,1:-1].to_numpy())
testTensor

In [ ]:
pictureCsv = pd.read_csv("../input/mnist-zeroshot-learning/trainPictureArray.csv")
sigmoidTrain = pd.read_csv("../input/mnist-zeroshot-learning/pictureIndexData.csv").iloc[:,1:]
sigmoidTrainV2 = pd.read_csv("../input/trainingmnistv3/newTraining (1).csv")
sigmoidTrainV2.columns = sigmoidTrain.columns
sigmoidTrain = pd.concat([sigmoidTrain,sigmoidTrainV2]).reset_index(drop=True)

In [ ]:
sigmoidTrain

In [ ]:
print(sigmoidTrain.shape)

In [ ]:
sigmoidTrain = sigmoidTrain.sample(frac=1,random_state=42).reset_index(drop=True)

In [ ]:
sigmoidTrain = sigmoidTrain.iloc[:,:]

In [ ]:
train = sigmoidTrain 
validation = pd.read_csv("../input/validationmnist/newvalidationset (1).csv")

In [ ]:
pictureCsvNoLabel = pictureCsv.iloc[:,:-1]
pictureCsvNoLabel = tf.convert_to_tensor(pictureCsvNoLabel.to_numpy())

In [ ]:
pictureVal = pd.read_csv("../input/mnist-zeroshot-learning/testMnist.csv")

In [ ]:
picturevalidationNoLabel = pictureVal.iloc[:,:-1]
picturevalidationNoLabel = tf.convert_to_tensor(picturevalidationNoLabel.to_numpy())

In [ ]:
firstPictureIndex = train["firstPictureIndex"].values
secondPictureIndex = train["secondPictureIndex"].values

firstPictureIndexValidation = validation["ImageA"].values
secondPictureIndexValidation = validation["ImageB"].values

In [ ]:
image = pictureCsv.iloc[0][:-1].to_numpy()
imageV2 = np.stack([image,image,image])

In [ ]:
pixels = image.reshape((28, 28))

        # Plot
plt.title('Label is {label}'.format(label=2))
plt.imshow(pixels, cmap='gray')
plt.show()

In [ ]:
from tensorflow.keras.layers import *

In [ ]:
def preprocess_image(index,process_type = "train",target_shape = (28,28)):
    """
    Load the specified file as a JPEG image, preprocess it and
    resize it to the target shape.
    """
    image = tf.reshape(pictureCsvNoLabel[index],(28,28))
    image = tf.stack([image],axis=2)
    #image = tf.image.decode_jpeg(image_string, channels=3)
    image = tf.reshape(image,(28,28,1))

    image = tf.image.convert_image_dtype(image, tf.float32)
    image = tf.image.resize(image, target_shape)
    
    #if process_type == "train":
    #    image = process_Augment_data(image,img_size = target_shape[0])
        
    return image

def aug_fn(image, img_size):
    data = {"image":image}
    aug_data = transforms(image=image)
    aug_img = aug_data["image"]
    aug_img = tf.image.resize(aug_img, size=[img_size, img_size])
    return aug_img

def process_Augment_data(image, img_size):
    aug_img = tf.numpy_function(func=aug_fn, inp=[image, img_size], Tout=tf.float64)
    return aug_img

def preprocess_siamese(positive, negative):
    """
    Given the filenames corresponding to the three images, load and
    preprocess them.
    """

    return (
        preprocess_image(positive),
        preprocess_image(negative),
    )

In [ ]:
imageFinalize = preprocess_image(0)

In [ ]:
image = pictureCsv.iloc[0][:-1].to_numpy().reshape((28,28))
imageV2 = np.stack([image,image,image],axis=2)
#imageV2 = imageV2.reshape((28,28,3))
#imageV2 = np.swapaxes(imageV2,0,2)
resized = cv2.resize(imageV2, (224,224), interpolation = cv2.INTER_AREA)
resized = resized*255

In [ ]:
import cv2

In [ ]:
for i in range(1):  
  plt.subplot(330 + 1 + i)
  plt.imshow(imageFinalize.numpy() , cmap=plt.get_cmap('gray'))
  plt.show()

Add Data Augmentation

In [ ]:
sigmoidTrain["symmetrylabel"].values

In [ ]:
def create_dataset(listFirstPictureIndex,listSecondPictureIndex,batchsize,labels,useType="train"):
    FirstIndex = listFirstPictureIndex
    SecondIndex = listSecondPictureIndex
    dataset1 = tf.data.Dataset.from_tensor_slices(FirstIndex)
    dataset2 = tf.data.Dataset.from_tensor_slices(SecondIndex)
    labelDatasets = tf.data.Dataset.from_tensor_slices(labels)
    
    dataset = tf.data.Dataset.zip((dataset1, dataset2))
    dataset = dataset.map(lambda x,y:preprocess_siamese(x,y))
    
    dataset = tf.data.Dataset.zip((dataset,labelDatasets))
    dataset = dataset.batch(batchsize, drop_remainder=False).prefetch(8)
    
    return dataset

In [ ]:
def preprocess_siamese_test(positive, negative,mode="validation"):
    """
    Given the filenames corresponding to the three images, load and
    preprocess them.
    """

    return (
        preprocess_image_test(positive,mode),
        preprocess_image_test(negative,mode),
    )

def create_dataset_Similarity(listFirstPictureIndex,listSecondPictureIndex,batchsize,labels,useType="train"):
    FirstIndex = listFirstPictureIndex
    SecondIndex = listSecondPictureIndex
    dataset1 = tf.data.Dataset.from_tensor_slices(FirstIndex)
    dataset2 = tf.data.Dataset.from_tensor_slices(SecondIndex)
    labelDatasets = tf.data.Dataset.from_tensor_slices(labels)
    
    dataset = tf.data.Dataset.zip((dataset1, dataset2))
    dataset = dataset.map(lambda x,y:preprocess_siamese_test(x,y,useType))
    
    dataset = tf.data.Dataset.zip((dataset,labelDatasets))
    dataset = dataset.batch(batchsize, drop_remainder=False).prefetch(8)
    
    return dataset

In [ ]:
def preprocess_image_test(index,process_type = "train",target_shape = (28,28)):
    """
    Load the specified file as a JPEG image, preprocess it and
    resize it to the target shape.
    """
    image = tf.reshape(testTensor[index],(28,28))
    image = tf.stack([image],axis=2)
    #image = tf.image.decode_jpeg(image_string, channels=3)
    image = tf.reshape(image,(28,28,1))

    image = tf.image.convert_image_dtype(image, tf.float32)
    image = tf.image.resize(image, target_shape)
    
    return image


In [ ]:
train_ds = create_dataset(firstPictureIndex,secondPictureIndex,128,train["symmetrylabel"].values,"train")
val_ds = create_dataset_Similarity(firstPictureIndexValidation,secondPictureIndexValidation,128,validation["label"].values,"validation")

In [ ]:
for data in train_ds.take(3):
  print(data[0][0].shape)

In [ ]:
for data in val_ds.take(3):
  print(data[0][0].shape)

In [ ]:
inp = Input(shape = (784))  
x = Dense(400, activation="relu")(inp)
x = Dense(200, activation="relu")(x)
x = Dense(100,activation="relu")(x)
x = Dense(50,activation = "relu")(x)
x = Dense(30,activation = "relu")(x)
#x = Dense(10,activation = "relu")(x)
#x = Dense(4,activation = "relu")(x)
#x = Dense(4,activation = "relu")(x)
#x = Dense(10,activation = "relu")(x)
x = Dense(30,activation = "relu")(x)
x = Dense(50,activation="relu")(x)
x = Dense(100,activation="relu")(x)
x = Dense(200, activation="relu")(x)
x = Dense(400, activation="relu")(x)
x = Dense(784,activation="relu")(x)
model = tf.keras.Model(inp,x)

In [ ]:
model.summary()

In [ ]:
trainPictureArray = pd.read_csv("../input/mnist-zeroshot-learning/trainPictureArray.csv")
encoderPictureIndex = trainPictureArray.copy()
testPictureArray = pd.read_csv("../input/mnist-zeroshot-learning/testMnist.csv")

In [ ]:
fullTrain = pd.concat([trainPictureArray,testPictureArray]).iloc[:,:-2].reset_index(drop=True)
fullTrain

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_validation, y_train, y_validation = train_test_split(fullTrain,fullTrain, test_size=0.2, random_state=42)

In [ ]:
trainingIndex = X_train.index
valIndex = X_validation.index
valIndex

In [ ]:
pictureCsvNoLabel = tf.convert_to_tensor(fullTrain.to_numpy())

In [ ]:
def preprocess_image_autoencoder(index,process_type = "train"):
    """
    Load the specified file as a JPEG image, preprocess it and
    resize it to the target shape.
    """
    image = pictureCsvNoLabel[index]
    return image

In [ ]:
def preprocess_image_test_autoencoder(index,process_type = "train"):
    """
    Load the specified file as a JPEG image, preprocess it and
    resize it to the target shape.
    """
    image = testTensor[index]
    return image

In [ ]:
def preprocess_autoencoder(imageIndexs,mode="validation"):
    """
    Given the filenames corresponding to the three images, load and
    preprocess them.
    """
    if mode == "test":
        return preprocess_image_test_autoencoder(imageIndexs,mode)
    else:
        return preprocess_image_autoencoder(imageIndexs,mode)

def create_autoencoder_dataset(trainingIndex,batchsize,useType="train"):
    
    dataset1 = tf.data.Dataset.from_tensor_slices(trainingIndex)
    #dataset2 = tf.data.Dataset.from_tensor_slices(SecondIndex)
    #labelDatasets = tf.data.Dataset.from_tensor_slices(labels)
    dataset1 = dataset1.map(lambda x:preprocess_autoencoder(x,useType))
    dataset = tf.data.Dataset.zip((dataset1, dataset1))
    #dataset = dataset.map(lambda x:preprocess_autoencoder(x,useType))
    if useType == "train":
        dataset = dataset.shuffle(1024)
    #dataset = tf.data.Dataset.zip((dataset,labelDatasets))
    dataset = dataset.batch(batchsize, drop_remainder=False).prefetch(8)
    
    return dataset

In [ ]:
train_ds = create_autoencoder_dataset(trainingIndex,64)
val_ds = create_autoencoder_dataset(valIndex,64)
#test_ds = create_autoencoder_dataset(testIndex,64,"test")

In [ ]:
model.compile(optimizer="Adam", loss="mse", metrics=["mae"])

In [ ]:
checkpoint_filepath = "./autoEncoderModel{epoch:02d}-validationloss{val_loss:.4f}.h5"
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_freq = "epoch",
    save_weights_only=True,
    monitor='val_loss',
    save_best_only=False,
    verbose = 1
    )

In [ ]:
history = model.fit(train_ds, epochs=100, validation_data=val_ds,callbacks=[model_checkpoint_callback])

1. autoEncoder MSE loss 0.02 mnist validation loss 0.16 smallest autoencoder layer 10
2. autoEncoder MSE loss 0.0121 mnist validation loss 0.149 smallest autoencoder layer 20
3. autoEncoder MSE loss 0.0088 mnist validation smallest autoencoder layer 30

In [ ]:
model.load_weights("./autoEncoderModel99-validationloss0.0088.h5")

In [ ]:
predict_ds = create_autoencoder_dataset(fullTrain.index.values,64,"test")

In [ ]:
label = pd.concat([trainPictureArray,testPictureArray]).iloc[:,-2].values

In [ ]:
fullTrain

In [ ]:
predict_ds = train_ds

In [ ]:
for data,_ in train_ds:
    new_data = model.predict(data)
    break

In [ ]:
def visualizeNumber(firstPicture,secondPicture):
    fig, (ax1, ax2) = plt.subplots(1, 2)
    fig.suptitle('Reconstruction Picture')
    ax1.imshow(firstPicture)
    ax2.imshow(secondPicture)
    ax1.set_title('Before Autoencoder')
    ax2.set_title('After Autoencoder')
    plt.show()

In [ ]:
def create_autoencoder_dataset_test(trainingIndex,batchsize,useType="train"):
    
    dataset1 = tf.data.Dataset.from_tensor_slices(trainingIndex)
    #dataset2 = tf.data.Dataset.from_tensor_slices(SecondIndex)
    #labelDatasets = tf.data.Dataset.from_tensor_slices(labels)
    dataset1 = dataset1.map(lambda x:preprocess_autoencoder(x,useType))
    dataset = tf.data.Dataset.zip((dataset1, dataset1))
    #dataset = dataset.map(lambda x:preprocess_autoencoder(x,useType))
    #dataset = tf.data.Dataset.zip((dataset,labelDatasets))
    dataset = dataset.batch(batchsize, drop_remainder=False).prefetch(8)
    
    return dataset

In [ ]:
predict_ds = create_autoencoder_dataset_test(fullTrain.index.values,64,"test")

In [ ]:
label[:100]

In [ ]:
fullTrain

In [ ]:
fullTrain = pd.concat([trainPictureArray,testPictureArray]).iloc[:,:-2].reset_index(drop=True)
label = pd.concat([trainPictureArray,testPictureArray]).iloc[:,-2].reset_index(drop=True).values

In [ ]:
label

In [ ]:
inferenceData = tf.data.Dataset.from_tensor_slices((fullTrain,label))
predict_ds = inferenceData.batch(64, drop_remainder=False).prefetch(8)

In [ ]:
newDataIncludeTest = np.zeros(fullTrain.shape)
labelIncludeTest = np.zeros(fullTrain.shape[0])
labelIncludeTest

In [ ]:
from tqdm.notebook import tqdm

In [ ]:
start = 0
for data,_ in tqdm(predict_ds):
    #new_data = model.predict(data)
    newDataIncludeTest[start:start+len(data)] = model.predict(data)
    start = start+len(data)
    

In [ ]:
start = 0
for _,label in tqdm(predict_ds):
    #new_data = model.predict(data)
    #newDataIncludeTest[start:start+len(data)] = model.predict(data)
    labelIncludeTest[start:start+len(label)] = label.numpy()
    start = start+len(label)

In [ ]:
labelIncludeTest

In [ ]:
 newDataIncludeTest

In [ ]:
autoEncodedData = pd.DataFrame(newDataIncludeTest,columns = [f"Pixel{i}" for i in range(28*28)])
autoEncodedData["label"] = labelIncludeTest

In [ ]:
autoEncodedData

In [ ]:
autoEncodedData

In [ ]:
import random

In [ ]:
indexViz = random.randint(0,69999)
print(f'label = {autoEncodedData.iloc[indexViz,-1]}')
visualizeNumber(fullTrain.to_numpy()[indexViz].reshape((28,28)),autoEncodedData.to_numpy()[indexViz ][:-1].reshape((28, 28)))

In [ ]:
autoEncodedData["label"] = labelIncludeTest

In [ ]:
trainOnly = autoEncodedData[autoEncodedData["label"] < 5]
testOnly = autoEncodedData[autoEncodedData["label"] >= 5]

In [ ]:
trainOnly.to_csv("newTrainPictureArray.csv",index=False)

In [ ]:
testOnly.to_csv("newTestPictureArray.csv",index=False)

In [ ]:
model.

In [ ]:
checkpoint_filepath = "./siamesemodelNodense148xxxparams{epoch:02d}-validationloss{val_loss:.2f}.h5"
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_freq = "epoch",
    save_weights_only=True,
    monitor='val_loss',
    save_best_only=False,
    verbose = 1
    )

In [ ]:
 siamese_network.fit(train_ds, epochs=100, validation_data=val_ds,callbacks=[model_checkpoint_callback])

* 10000 training best val_loss =  0.1916,train_loss = 0.0030
* 20000 training best val_loss =  0.1838,train_loss = 0.0017
* 30000 training best val_loss =  0.1595,train_loss = 8.8381e-04
* 40000 training best val_loss =  0.1716
* 46000 training best val_loss = 0.172
* 246000 training best val_loss = 

In [ ]:
val_distance = []
val_trueLabel = []
for data,labels in tqdm(val_ds):
    preds = siamese_network.predict(data)
    for eachPred in preds:
        val_distance.append(eachPred[0])
    for eachLabel in labels:
        val_trueLabel.append(eachLabel.numpy())

In [ ]:
valDf = pd.DataFrame({})

In [ ]:
siamese_network.load_weights("../input/version1/siamesemodel36-validationloss0.16.h5")

In [ ]:
siamese_network

In [ ]:
def preprocess_siamese_test(positive, negative,mode="validation"):
    """
    Given the filenames corresponding to the three images, load and
    preprocess them.
    """

    return (
        preprocess_image_test(positive,mode),
        preprocess_image_test(negative,mode),
    )

def create_dataset_Similarity(listFirstPictureIndex,listSecondPictureIndex,batchsize,labels,useType="train"):
    FirstIndex = listFirstPictureIndex
    SecondIndex = listSecondPictureIndex
    dataset1 = tf.data.Dataset.from_tensor_slices(FirstIndex)
    dataset2 = tf.data.Dataset.from_tensor_slices(SecondIndex)
    labelDatasets = tf.data.Dataset.from_tensor_slices(labels)
    
    dataset = tf.data.Dataset.zip((dataset1, dataset2))
    dataset = dataset.map(lambda x,y:preprocess_siamese_test(x,y,useType))
    
    dataset = tf.data.Dataset.zip((dataset,labelDatasets))
    dataset = dataset.batch(batchsize, drop_remainder=False).prefetch(8)
    
    return dataset

In [ ]:
def create_dataset_Similarity(listFirstPictureIndex,listSecondPictureIndex,batchsize,labels,useType="train"):
    FirstIndex = listFirstPictureIndex
    SecondIndex = listSecondPictureIndex
    dataset1 = tf.data.Dataset.from_tensor_slices(FirstIndex)
    dataset2 = tf.data.Dataset.from_tensor_slices(SecondIndex)
    labelDatasets = tf.data.Dataset.from_tensor_slices(labels)
    
    dataset = tf.data.Dataset.zip((dataset1, dataset2))
    dataset = dataset.map(lambda x,y:preprocess_siamese_test(x,y,useType))
    
    dataset = tf.data.Dataset.zip((dataset,labelDatasets))
    dataset = dataset.batch(batchsize, drop_remainder=False).prefetch(8)
    
    return dataset

In [ ]:
testData = pd.read_csv("../input/mnist-zeroshot-learning/testMnist.csv")
testPictureIndex = testData.index.values
testTensor = tf.convert_to_tensor(testData.iloc[:,1:-1].to_numpy())
testTensor

In [ ]:
testTensor = tf.convert_to_tensor(testData.iloc[:,1:-1].to_numpy())
testTensor

In [ ]:
testSimilarity = pd.read_csv("../input/validationmnist/newTestIndex.csv")

In [ ]:
testSimilarity

In [ ]:
def preprocess_image_test(index,process_type = "train",target_shape = (224,224)):
    """
    Load the specified file as a JPEG image, preprocess it and
    resize it to the target shape.
    """
    image = tf.reshape(testTensor[index],(28,28))
    image = tf.stack([image],axis=2)
    #image = tf.image.decode_jpeg(image_string, channels=3)
    image = tf.reshape(image,(28,28,1))

    image = tf.image.convert_image_dtype(image, tf.float32)
    
    return image


In [ ]:
testdsv2 = create_dataset_Similarity(testSimilarity["ImageA"],testSimilarity["ImageB"],64,testSimilarity["label"].values,"validation")

In [ ]:
from tqdm.notebook import tqdm

In [ ]:
distance = []
trueLabel = []
for data,labels in tqdm(testdsv2):
    preds = siamese_network.predict(data)
    for eachPred in preds:
        distance.append(eachPred[0])
    for eachLabel in labels:
        trueLabel.append(eachLabel.numpy())
        

In [ ]:
predictionDf = pd.DataFrame({"Label":trueLabel,"distancePred":distance})

In [ ]:
siamese_network.load_weights("../input/version1/siamesemodel04-validationloss0.16.h5")

In [ ]:
distance = []
trueLabel = []
for data,labels in tqdm(testdsv2):
    preds = siamese_network.predict(data)
    for eachPred in preds:
        distance.append(eachPred[0])
    for eachLabel in labels:
        trueLabel.append(eachLabel.numpy())

In [ ]:
def testPrediction(dataset):
    distance = []
    trueLabel = []
    for data,labels in tqdm(dataset):
        preds = siamese_network.predict(data)
        for eachPred in preds:
            distance.append(eachPred[0])
        for eachLabel in labels:
            trueLabel.append(eachLabel.numpy()) 
    return trueLabel

In [ ]:
siamese_network.load_weights("../input/version1/siamesemodelNoDense02-validationloss0.16.h5")

In [ ]:
predictionDf

In [ ]:
predictionDf["Another0.16V2"] = testPrediction(testdsv2)

In [ ]:
siamese_network.load_weights("../input/version1/siamesemodelNodense148xxxparams02-validationloss0.17.h5")
predictionDf["Another0.17"] = testPrediction(testdsv2)

In [ ]:
predictionDf.to_csv("TestPredEnsemblePrepared.csv",index=False)

In [ ]:
predictionDf["Another0.16"] = distance

In [ ]:
predictionDf["Ensembling"] = (predictionDf["Another0.16"] + predictionDf["distancePred"])/2

In [ ]:
predictionDf

In [ ]:
predictionDf["labelPred"] = predictionDf["Ensembling"].apply(lambda x: 1 if x < 0.5 else 0)

In [ ]:
(predictionDf["Label"] == predictionDf["labelPred"]).sum()/170000 * 100

15% of testing data is validation set

In [ ]:
from sklearn.model_selection import train_test_split
X_test, X_validation, y_test, y_validation = train_test_split(testSimilarity, testSimilarity, test_size=0.15, random_state=42,shuffle=True)

In [ ]:
x_val = X_validation.reset_index(drop=True)
y_val = x_val["label"].values
#x_val.drop(columns="label",inplace=True)

In [ ]:
x_val.to_csv("newvalidationset.csv",index=False)

In [ ]:
X_test.reset_index(drop=True,inplace=True)

In [ ]:
X_test.to_csv("newTestIndex.csv",index=False)